In [1]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import geemap

import os
import geopandas as gpd

from shapely import wkt
import pandas as pd
import matplotlib.pyplot as plt

### Funciones

In [2]:
def calcular_distancia(imagen_pixel, nom_vars_list, df_sh_list):
    
    """Funcion para calcular la distancia de la imagen con diferentes sectores (medido en metros)"""
    
    for i in range(len(df_sh_list)):
        print('revisando ' + str(i) + '----')
        df_sh_list[i].to_crs(epsg=32719, inplace=True)                # to meters
        imagen_pixel[nom_vars_list[i]] = imagen_pixel.geometry.apply(lambda x: df_sh_list[i].distance(x).min())  # add distance colums
    
    return imagen_pixel

def add_time(image):
    """Funcion para realizar un filtro de fechas con map"""
    return image.set('Date', image.date().format("YYYY-MM-dd"))



def calculate_weather(image, fecha, data_con_dependiente_4326):
    
    # filtro df
    df_aux = data_con_dependiente_4326[data_con_dependiente_4326['time2'] == fecha] # global env
    
    # recolectamos los puntos
    points = df_aux.geometry

    # formato apto para buscar los puntos en gee
    jsonDict = eval(points.to_json()) 
    fc = ee.FeatureCollection(jsonDict)
    imagen_per_point = image.sampleRegions(collection = fc)
    
    fc_point_samp = geemap.ee_to_df(imagen_per_point)
    df_aux = df_aux.reset_index(drop= True)
    fc_point_samp = fc_point_samp.reset_index(drop= True)
    
    return pd.concat([df_aux, fc_point_samp], axis=1)


### Estimando distancias

In [3]:
# load data pixeles
data_con_dependiente = pd.read_csv('data_con_dependiente.csv')

# editando geometria
data_con_dependiente['geometry'] = data_con_dependiente['geometry'].apply(wkt.loads)
data_con_dependiente = gpd.GeoDataFrame(data_con_dependiente, geometry='geometry')
data_con_dependiente.crs = "EPSG:32719"

In [4]:
# load .shp
lista_gdfs = []

for directorio_raiz, directorios, archivos in os.walk('shp'):
    for archivo in archivos:
        if archivo.endswith('.shp'):            # Solo considerar archivos con extensión .shp
            ruta_archivo = os.path.join(directorio_raiz, archivo)
            print(ruta_archivo)
            gdf = gpd.read_file(ruta_archivo)   # Cargar el Shapefile
            lista_gdfs.append(gdf)   

shp\areas_silverstres_protegidas\snaspe.shp
shp\areas_urbanas\Areas_Pobladas.shp
shp\masas_lacustres\masas_lacustres.shp
shp\Masas_lacustres_sector_sur\Masas_lacustres_sector_sur.shp
shp\red_ferroviaria\Red_ferroviaria.shp
shp\red_hidrografica\Red_Hidrografica.shp
shp\red_vial\redvial2019.shp


NO SIRVEN:

- shp\masas_lacustres\masas_lacustres.shp
- shp\red_ferroviaria\Red_ferroviaria.shp 


In [5]:
areas_silvestres_protegidas = lista_gdfs[0][lista_gdfs[0]['Region'] == 'Región de Valparaíso']
red_hidrografica = lista_gdfs[5][lista_gdfs[5]['Region'] == 'Región de Valparaíso']

In [6]:
# AREAS URBANAS VALPO

aux = red_hidrografica['Comuna'].unique()  # no esta rapa nui ni juan fernandez!
aux[1] = 'La Calera'

areas_urbanas = lista_gdfs[1][lista_gdfs[1]['comuna'].isin(aux)]

In [7]:
red_vial_2019 = lista_gdfs[6][lista_gdfs[6]['Cod_Region']==5]

#### Calculando distancias:

In [8]:
nom_vars_list = ['d_areas_silvestres_protegidas', 'd_red_hidrografica', 'd_areas_urbanas', 'd_red_vial', 'd_masas_lacustres_sur']
df_sh_list = [areas_silvestres_protegidas, red_hidrografica, areas_urbanas, red_vial_2019, lista_gdfs[3]]

df_dist = calcular_distancia(data_con_dependiente, nom_vars_list, df_sh_list)
df_dist

revisando 0----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 1----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 2----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 3----


c:\Users\jmpreussa\Desktop\dataton\.venv\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


revisando 4----


,time,dim_0,y,x,spatial_ref,coastal,blue,green,red,rededge1,...,scl,aot,wvp,geometry,es_incendio,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
0,2022-11-29 14:52:04.587,0,6327645.0,299625.0,32719,2226,2049,2027,1980,2217,...,8,229,765,POINT (299625.000 6327645.000),0,18244.225859,29.072234,18426.709083,303.226095,14910.575852
1,2022-11-29 14:52:04.587,1,6323535.0,299985.0,32719,7669,6870,6497,6323,6599,...,9,217,765,POINT (299985.000 6323535.000),0,22299.602802,1779.822053,22214.050841,622.617851,13543.860309
2,2022-11-29 14:52:04.587,2,6324855.0,298305.0,32719,5079,4679,4418,4264,4543,...,9,217,765,POINT (298305.000 6324855.000),0,20543.270239,319.798819,20309.635801,1469.251037,12425.131087
3,2022-11-29 14:52:04.587,3,6326835.0,300195.0,32719,7397,7002,6632,6590,6933,...,9,220,790,POINT (300195.000 6326835.000),0,18993.434741,1013.959086,19414.474463,1045.742944,14997.459742
4,2022-11-29 14:52:04.587,4,6322815.0,299055.0,32719,8905,7804,7218,6891,7126,...,9,213,765,POINT (299055.000 6322815.000),0,21473.648255,684.735900,22479.749207,511.911588,12456.298655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2022-12-24 14:52:03.923,95,6324435.0,299175.0,32719,727,856,938,1099,1190,...,5,105,1025,POINT (299175.000 6324435.000),1,21433.831207,751.459293,21051.695506,524.400935,13065.362539
596,2022-12-24 14:52:03.923,96,6323535.0,299955.0,32719,371,464,700,868,1239,...,5,106,1044,POINT (299955.000 6323535.000),0,22283.148427,1753.583381,22200.838958,597.995448,13515.073981
597,2022-12-24 14:52:03.923,97,6327285.0,303555.0,32719,593,848,913,1067,1105,...,5,119,932,POINT (303555.000 6327285.000),1,18284.154530,812.121098,20992.233688,799.775649,18210.049594
598,2022-12-24 14:52:03.923,98,6326415.0,302145.0,32719,695,714,754,942,1036,...,5,119,860,POINT (302145.000 6326415.000),1,19254.683885,732.140576,20830.178864,448.449895,16567.601912


In [9]:
del lista_gdfs
df_dist.describe()

,dim_0,y,x,spatial_ref,coastal,blue,green,red,rededge1,rededge2,...,swir22,scl,aot,wvp,es_incendio,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,d_masas_lacustres_sur
count,600.000000,6.000000e+02,600.000000,600.0,600.000000,600.00000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,49.500000,6.325083e+06,301296.900000,32719.0,1058.341667,1096.95500,1263.700000,1512.901667,1808.790000,2180.188333,...,2228.636667,5.256667,135.785000,1087.658333,0.340000,20495.471272,834.583748,21588.793813,900.688378,15376.074601
std,28.890155,1.731712e+03,2157.054983,0.0,1770.860146,1562.86936,1388.458574,1287.053061,1277.597274,1175.596522,...,773.654943,1.184537,40.889755,205.000855,0.474104,1578.386608,560.734317,1763.816035,680.484022,2159.030320
min,0.000000,6.322155e+06,297975.000000,32719.0,256.000000,204.00000,384.000000,305.000000,670.000000,920.000000,...,832.000000,2.000000,81.000000,462.000000,0.000000,18041.191966,3.605228,18286.246392,5.457055,11802.928598
25%,24.750000,6.323535e+06,299490.000000,32719.0,431.750000,537.00000,744.750000,969.750000,1241.750000,1641.000000,...,1785.250000,5.000000,111.750000,955.750000,0.000000,19086.231689,384.675261,20186.855948,428.132200,13572.978658
50%,49.500000,6.325125e+06,300735.000000,32719.0,500.000000,616.00000,867.000000,1170.000000,1474.500000,1875.500000,...,2052.500000,5.000000,125.000000,1099.500000,0.000000,20498.519584,785.915333,21619.650815,761.295432,15038.370912
75%,74.250000,6.326670e+06,303210.000000,32719.0,654.250000,805.00000,1013.750000,1419.750000,1720.000000,2160.000000,...,2419.000000,5.000000,143.000000,1219.250000,1.000000,21834.807706,1186.541111,22853.316550,1205.629051,17450.205464
max,99.000000,6.327645e+06,305055.000000,32719.0,10179.000000,9064.00000,8534.000000,8272.000000,8603.000000,8468.000000,...,6307.000000,9.000000,235.000000,1803.000000,1.000000,23459.737143,2681.883984,25301.435778,2940.895404,19588.010484


### Datos climáticos con Google Earth Engine

In [10]:
import ee
ee.Authenticate()

True

In [11]:
# Inicializar Earth Engine
ee.Initialize()

In [12]:
data_con_dependiente_4326 = data_con_dependiente.to_crs("EPSG:4326")
data_con_dependiente_4326['time2'] = [item.split(" ", 1)[0] for item in data_con_dependiente_4326['time']]

# Obtenemos las fechas de captura de cada uno, para realizar un posterior filtro en las imagenes recolectadas
DATES = data_con_dependiente_4326.time.unique()
DATES = [item.split(" ", 1)[0] for item in DATES]
DATES

inicio_datetime = min(DATES)
fin_datetime = max(DATES)

In [13]:
list_df = []

for fecha in DATES:
    print('Extrayendo info para el día: ' + fecha)
    
    # Definir la colección de imágenes
    image_collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(fecha) \
        .select([
            'temperature_2m',
            'temperature_2m_min',
            'temperature_2m_max',
            'dewpoint_temperature_2m',
            'dewpoint_temperature_2m_min',
            'dewpoint_temperature_2m_max',
            'soil_temperature_level_1',
            'soil_temperature_level_1_min',
            'soil_temperature_level_1_max',
            'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_1_min',
            'volumetric_soil_water_layer_1_max',
            'u_component_of_wind_10m',
            'u_component_of_wind_10m_min',
            'u_component_of_wind_10m_max',
            'v_component_of_wind_10m',
            'v_component_of_wind_10m_min',
            'v_component_of_wind_10m_max',
            'total_precipitation_sum',
            'total_precipitation_min',
            'total_precipitation_max'
        ])
        
    # Realizando estimaciones: 
    df_aux = calculate_weather(image_collection.first(), fecha, data_con_dependiente_4326)
              
    list_df.append(df_aux)
    
data_con_gee = pd.concat(list_df, ignore_index = True)
        

Extrayendo info para el día: 2022-11-29
Extrayendo info para el día: 2022-12-04
Extrayendo info para el día: 2022-12-09
Extrayendo info para el día: 2022-12-14
Extrayendo info para el día: 2022-12-19
Extrayendo info para el día: 2022-12-24


In [14]:
data_con_gee.to_csv('data_con_geee.csv')